# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-15-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-15-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-16 04:44:59,34.223334,-82.461707,165,1,0,164.0,"Abbeville, South Carolina, US",672.728014,0.606061
1,22001.0,Acadia,Louisiana,US,2020-07-16 04:44:59,30.295065,-92.414197,1510,46,0,1464.0,"Acadia, Louisiana, US",2433.717463,3.046358
2,51001.0,Accomack,Virginia,US,2020-07-16 04:44:59,37.767072,-75.632346,1045,14,0,1031.0,"Accomack, Virginia, US",3233.692289,1.339713
3,16001.0,Ada,Idaho,US,2020-07-16 04:44:59,43.452658,-116.241552,4836,28,0,4808.0,"Ada, Idaho, US",1004.179930,0.578991
4,19001.0,Adair,Iowa,US,2020-07-16 04:44:59,41.330756,-94.471059,17,0,0,17.0,"Adair, Iowa, US",237.695749,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,33190,33384,33594,33908,34194,34366,34451,34455,34740,34994
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2964,3038,3106,3188,3278,3371,3454,3571,3667,3752
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,16404,16879,17348,17808,18242,18712,19195,19689,20216,20770


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,898,920,936,957,971,994,1010,1012,1048,1094
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,79,81,83,83,85,89,93,95,97,101
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,959,968,978,988,996,1004,1011,1018,1028,1040


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,20103,20179,20700,20847,20882,21135,21216,21254,21454,22456
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1702,1744,1791,1832,1875,1881,1946,2014,2062,2091
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,11884,12094,12329,12637,13124,13124,13743,14019,14295,14792


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,301,303,307,309,310,310,310,312,312,313
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,31,31,31,31,31,33,33,33,36,36


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
3052,1131.0,Wilcox,Alabama,US,2020-07-16 04:44:59,31.987732,-87.308911,332,8,0,324.0,"Wilcox, Alabama, US",3200.616986,2.409639
521,1025.0,Clarke,Alabama,US,2020-07-16 04:44:59,31.680999,-87.835486,339,6,0,333.0,"Clarke, Alabama, US",1435.102870,1.769912
852,1051.0,Elmore,Alabama,US,2020-07-16 04:44:59,32.597854,-86.144153,1118,25,0,1093.0,"Elmore, Alabama, US",1376.694701,2.236136


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,3497847,137407,1075882
Brazil,1966748,75366,1350098
India,968857,24914,612768


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,3497847,137407,1075882,2284558,2020-07-16 04:44:59,38.337984,-91.781303
Brazil,1966748,75366,1350098,541284,2020-07-16 04:44:59,-12.669522,-48.480493
India,968857,24914,612768,331175,2020-07-16 04:44:59,23.356511,82.081064
Russia,745197,11753,522375,211069,2020-07-16 04:44:59,54.546312,62.120860
Peru,337751,12417,226400,98934,2020-07-16 04:44:59,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,404006,32427,0
California,354885,7375,0
Florida,301810,4521,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,404006,32427,0,371579,2020-07-16 04:44:59,42.671593,-75.579694
California,354885,7375,0,347510,2020-07-16 04:44:59,37.778248,-120.728666
Florida,301810,4521,0,297289,2020-07-16 04:44:59,28.940755,-82.700744
Texas,289837,3498,0,286339,2020-07-16 04:44:59,31.641665,-98.609412
New Jersey,176278,15634,0,160644,2020-07-16 04:44:59,40.286441,-74.622450


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,219982,23353,0
California,Los Angeles,143207,3943,0
Illinois,Cook,96662,4745,0
Arizona,Maricopa,86483,1277,0
Florida,Miami-Dade,72317,1202,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,219982,23353,0,196629,2020-07-16 04:44:59,40.767273,-73.971526,36061.0
California,Los Angeles,143207,3943,0,139264,2020-07-16 04:44:59,34.308284,-118.228241,6037.0
Illinois,Cook,96662,4745,0,91917,2020-07-16 04:44:59,41.841448,-87.816588,17031.0
Arizona,Maricopa,86483,1277,0,85206,2020-07-16 04:44:59,33.348359,-112.491815,4013.0
Florida,Miami-Dade,72317,1202,0,71115,2020-07-16 04:44:59,25.611236,-80.551706,12086.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,34455,3571,19689,858,525,74,103265,32151,10251,18948,...,291691,989,13591,9707,373,6566,10,1498,1895,1034
2020-07-14,34740,3667,20216,861,541,74,106910,32490,10487,19021,...,292931,997,14085,10010,373,6764,10,1516,1895,1064
2020-07-15,34994,3752,20770,862,576,74,111146,33005,10810,19154,...,293469,1009,14581,10428,381,7064,10,1526,1895,1089


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,1012,95,1018,52,26,3,1903,573,108,708,...,44915,31,64,93,0,39,1,424,42,19
2020-07-14,1048,97,1028,52,26,3,1968,581,111,709,...,45053,31,67,96,0,44,1,429,42,20
2020-07-15,1094,101,1040,52,27,3,2050,592,113,710,...,45138,31,71,100,0,44,1,433,42,20


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,21254,2014,14019,803,118,57,44173,19865,7835,17000,...,1384,903,8030,2671,352,1084,8,675,1412,343
2020-07-14,21454,2062,14295,803,118,57,45467,20729,7928,17073,...,1385,905,8327,2671,352,1084,8,685,1412,343
2020-07-15,22456,2091,14792,803,124,57,47298,21348,8035,17175,...,1386,909,8655,3050,353,1084,8,694,1412,395


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,1724,1770,1815,1866,1904,1926,1980,2050,2090,2133
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,233,236,241,245,267,275,290,309,332,355
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,312,317,320,330,342,352,354,361,364,381
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,199,199,202,206,212,215,219,222,227,235


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-13,55545,1539,0,123824,28939,333357,37218,47510,12879,49,...,65274,269778,30030,1301,206,71642,41757,4313,36942,1903
2020-07-14,57255,1579,0,128097,29812,346211,37668,47530,12969,49,...,66788,279577,30478,1305,206,72443,42304,4407,37906,1950
2020-07-15,59067,1631,0,131354,30297,354885,38137,47636,13050,49,...,69061,289837,30891,1318,243,73527,43046,4557,38727,1985


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-13         715    1341     402  227    331     378    655     554   
2020-07-14         733    1396     413  232    344     380    656     577   
2020-07-15         743    1499     426  238    367     381    659     642   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-13          685      131  ...      84      4       33       10   
2020-07-14          693      135  ...      84      4       33       10   
2020-07-15          699      140  ...      88      4       36       10   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-13            150   162   205          0       43      3  
2020-07-14            156   164   208          0       43      3  
2020-07-15            157   170   208          0       43      3  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-13,1124,17,0,2246,323,7089,1727,4371,517,0,...,749,3276,216,56,6,1968,1399,97,820,21
2020-07-14,1164,17,0,2337,331,7250,1738,4372,518,0,...,767,3362,226,56,6,1977,1404,97,826,22
2020-07-15,1211,17,0,2434,335,7375,1744,4380,521,0,...,783,3498,233,56,6,1992,1421,98,827,22


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-13          15      11       2    1      1      10     29       5   
2020-07-14          17      11       3    2      1      10     31       6   
2020-07-15          18      12       3    2      1      10     31       6   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-13           30        7  ...       0      0        0        0   
2020-07-14           32        7  ...       0      0        0        0   
2020-07-15           32        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-13              0     0     0         20        0      0  
2020-07-14              0     0     0         21        0      0  
2020-07-15              0     0     0         21        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,0.000116,0.033874,0.025736,0.003509,0.037549,0.0,0.030939,0.005693,0.027154,0.002699,...,0.001844,0.002026,0.045703,0.025568,0.002688,0.053933,0.0,0.022526,0.0,0.049746
2020-07-14,0.008272,0.026883,0.026766,0.003497,0.030476,0.0,0.035298,0.010544,0.023022,0.003853,...,0.004251,0.008089,0.036348,0.031215,0.000000,0.030155,0.0,0.012016,0.0,0.029014
2020-07-15,0.007311,0.023180,0.027404,0.001161,0.064695,0.0,0.039622,0.015851,0.030800,0.006992,...,0.001837,0.012036,0.035215,0.041758,0.021448,0.044352,0.0,0.006596,0.0,0.023496


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,0.001980,0.021505,0.006924,0.0,0.000000,0.0,0.031436,0.014159,0.000000,0.000000,...,0.000245,0.0,0.066667,0.044944,NaN,0.083333,0.0,0.016787,0.0,0.055556
2020-07-14,0.035573,0.021053,0.009823,0.0,0.000000,0.0,0.034157,0.013962,0.027778,0.001412,...,0.003072,0.0,0.046875,0.032258,NaN,0.128205,0.0,0.011792,0.0,0.052632
2020-07-15,0.043893,0.041237,0.011673,0.0,0.038462,0.0,0.041667,0.018933,0.018018,0.001410,...,0.001887,0.0,0.059701,0.041667,NaN,0.000000,0.0,0.009324,0.0,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,0.001791,0.034943,0.020083,0.0,0.000000,0.0,0.034642,0.011817,0.008495,0.002832,...,0.004354,0.007812,0.022669,0.000000,0.005714,0.150743,0.0,0.024279,0.0,0.045732
2020-07-14,0.009410,0.023833,0.019688,0.0,0.000000,0.0,0.029294,0.043494,0.011870,0.004294,...,0.000723,0.002215,0.036986,0.000000,0.000000,0.000000,0.0,0.014815,0.0,0.000000
2020-07-15,0.046705,0.014064,0.034767,0.0,0.050847,0.0,0.040271,0.029862,0.013496,0.005974,...,0.000722,0.004420,0.039390,0.141894,0.002841,0.000000,0.0,0.013139,0.0,0.151603


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-13,0.036539,0.040568,NaN,0.011081,0.020164,0.027158,0.008755,0.004716,0.005858,0.0,...,0.053486,0.026701,0.018519,0.003858,0.138122,0.013754,0.027081,0.016258,0.013554,0.022019
2020-07-14,0.030786,0.025991,NaN,0.034509,0.030167,0.038559,0.012091,0.000421,0.006988,0.0,...,0.023195,0.036322,0.014918,0.003075,0.000000,0.011181,0.013100,0.021795,0.026095,0.024698
2020-07-15,0.031648,0.032932,NaN,0.025426,0.016269,0.025054,0.012451,0.002230,0.006246,0.0,...,0.034033,0.036698,0.013551,0.009962,0.179612,0.014963,0.017540,0.034037,0.021659,0.017949


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-13      0.024355  0.050117  0.007519  0.013393  0.034375  0.000000   
2020-07-14      0.025175  0.041014  0.027363  0.022026  0.039275  0.005291   
2020-07-15      0.013643  0.073782  0.031477  0.025862  0.066860  0.002632   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-13      0.004601  0.075728  0.011817  0.056452  ... -0.011765    0.0   
2020-07-14      0.001527  0.041516  0.011679  0.030534  ...  0.000000    0.0   
2020-07-15      0.004573  0.112652  0.008658  0.037037  ...  0.047619    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-13      0.137931  0.428571   0.086957  0.031847  0.014851        NaN   
2020-07-14      0.000000  0.000000   0.040000  0.012346  0.014634        NaN   
2020-07-15      0.090909  0.000000   0.006410  0.036585  0.000000        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-07-13          0.0    2.0  
2020-07-14          0.0    0.0  
2020-07-15          0.0    0.0  

[3 rows x 3254 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-13,0.002676,0.0,NaN,0.004023,0.006231,0.005389,0.001159,0.005290,0.000000,NaN,...,0.010796,0.018657,0.004651,0.0,0.0,0.001017,-0.027121,0.010417,0.000000,0.000000
2020-07-14,0.035587,0.0,NaN,0.040516,0.024768,0.022711,0.006369,0.000229,0.001934,NaN,...,0.024032,0.026252,0.046296,0.0,0.0,0.004573,0.003574,0.000000,0.007317,0.047619
2020-07-15,0.040378,0.0,NaN,0.041506,0.012085,0.017241,0.003452,0.001830,0.005792,NaN,...,0.020860,0.040452,0.030973,0.0,0.0,0.007587,0.012108,0.010309,0.001211,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb Blount Bullock    Butler   
2020-07-13      0.000000  0.000000     0.0  0.0    0.0     0.0  0.000000   
2020-07-14      0.133333  0.000000     0.5  1.0    0.0     0.0  0.068966   
2020-07-15      0.058824  0.090909     0.0  0.0    0.0     0.0  0.000000   

Province_State                             ... Wyoming                  \
Admin2         Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2020-07-13         0.0  0.000000      0.0  ...     NaN    NaN      NaN   
2020-07-14         0.2  0.066667      0.0  ...     NaN    NaN      NaN   
2020-07-15         0.0  0.000000      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-13          NaN        NaN   NaN   NaN       0.00      NaN    NaN  
2020-07-14          NaN        NaN   NaN   NaN       0.05      NaN    NaN  
2020-07-15          NaN        NaN   NaN   NaN       0.00      NaN    NaN  

[3 rows x 3254 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,0.002332,0.029974,0.025777,0.001754,0.055276,0.002307,0.031735,0.011636,0.024295,0.003993,...,0.002083,0.002215,0.042392,0.031826,0.003019,0.055641,2.649095e-08,0.028208,0.001264,0.037759
2020-07-14,0.005302,0.028428,0.026272,0.002625,0.042876,0.001154,0.033516,0.011090,0.023658,0.003923,...,0.003167,0.005152,0.039370,0.031520,0.001509,0.042898,1.324548e-08,0.020112,0.000632,0.033386
2020-07-15,0.006307,0.025804,0.026838,0.001893,0.053786,0.000577,0.036569,0.013470,0.027229,0.005457,...,0.002502,0.008594,0.037292,0.036639,0.011479,0.043625,6.622738e-09,0.013354,0.000316,0.028441


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,0.010330,0.030244,0.007329,7.304491e-11,0.037907,8.141045e-28,0.026603,0.015735,0.001795,0.000711,...,0.000858,0.012788,0.064305,0.043271,NaN,0.109488,0.0,0.046083,0.003163,0.104265
2020-07-14,0.022952,0.025648,0.008576,3.652245e-11,0.018954,4.070522e-28,0.030380,0.014848,0.014787,0.001062,...,0.001965,0.006394,0.055590,0.037764,NaN,0.118847,0.0,0.028938,0.001581,0.078448
2020-07-15,0.033422,0.033443,0.010125,1.826123e-11,0.028708,2.035261e-28,0.036023,0.016891,0.016402,0.001236,...,0.001926,0.003197,0.057646,0.039716,NaN,0.059423,0.0,0.019131,0.000791,0.039224


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-13,0.004466,0.029490,0.025718,0.000117,0.001789,0.046207,0.039587,0.017792,0.007080,0.003457,...,0.002212,0.006382,0.022725,0.009798,0.003664,0.268180,1.552204e-10,0.024093,0.011873,0.064653
2020-07-14,0.006938,0.026661,0.022703,0.000059,0.000894,0.023103,0.034441,0.030643,0.009475,0.003876,...,0.001467,0.004299,0.029855,0.004899,0.001832,0.134090,7.761021e-11,0.019454,0.005937,0.032327
2020-07-15,0.026821,0.020363,0.028735,0.000029,0.025871,0.011552,0.037356,0.030252,0.011486,0.004925,...,0.001095,0.004359,0.034623,0.073397,0.002336,0.067045,3.880511e-11,0.016296,0.002968,0.091965


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-13,0.033641,0.047902,NaN,0.018673,0.023070,0.025139,0.009842,0.002586,0.006033,5.122154e-37,...,0.037844,0.031585,0.020963,0.005770,0.112301,0.013037,0.024762,0.024063,0.018307,0.019431
2020-07-14,0.032213,0.036947,NaN,0.026591,0.026618,0.031849,0.010966,0.001503,0.006511,2.561077e-37,...,0.030519,0.033954,0.017941,0.004422,0.056151,0.012109,0.018931,0.022929,0.022201,0.022064
2020-07-15,0.031931,0.034939,NaN,0.026008,0.021444,0.028452,0.011709,0.001867,0.006378,1.280539e-37,...,0.032276,0.035326,0.015746,0.007192,0.117881,0.013536,0.018235,0.028483,0.021930,0.020007


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-13      0.024966  0.050891  0.015099  0.017660  0.048698  0.002130   
2020-07-14      0.025070  0.045953  0.021231  0.019843  0.043986  0.003710   
2020-07-15      0.019356  0.059867  0.026354  0.022853  0.055423  0.003171   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-13      0.004486  0.066731  0.010298  0.042739  ...  0.006323   
2020-07-14      0.003006  0.054123  0.010989  0.036636  ...  0.003162   
2020-07-15      0.003790  0.083388  0.009823  0.036837  ...  0.025390   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-13      0.000071  0.073599  0.235121   0.057750  0.024317  0.011223   
2020-07-14      0.000036  0.036800  0.117560   0.048875  0.018331  0.012928   
2020-07-15      0.000018  0.063854  0.058780   0.027643  0.027458  0.006464   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-07-13            NaN  0.002895  0.998077  
2020-07-14            NaN  0.001447  0.499039  
2020-07-15            NaN  0.000724  0.249519  

[3 rows x 3254 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-13,0.007043,0.000589,NaN,0.019454,0.008849,0.006946,0.001424,0.002703,0.000185,NaN,...,0.012009,0.022169,0.011023,2.709303e-10,3.523713e-20,0.002815,-0.009859,0.006576,0.001366,0.000391
2020-07-14,0.021315,0.000294,NaN,0.029985,0.016808,0.014828,0.003897,0.001466,0.001060,NaN,...,0.018020,0.024210,0.028660,1.354652e-10,1.761856e-20,0.003694,-0.003143,0.003288,0.004342,0.024005
2020-07-15,0.030846,0.000147,NaN,0.035746,0.014446,0.016035,0.003674,0.001648,0.003426,NaN,...,0.019440,0.032331,0.029817,6.773258e-11,8.809282e-21,0.005641,0.004483,0.006799,0.002776,0.012002


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin   Barbour  Bibb Blount   Bullock    Butler   
2020-07-13      0.025292  0.009722  0.000488  0.00    0.0  0.000106  0.002250   
2020-07-14      0.079313  0.004861  0.250244  0.50    0.0  0.000053  0.035608   
2020-07-15      0.069068  0.047885  0.125122  0.25    0.0  0.000026  0.017804   

Province_State                                        ... Wyoming         \
Admin2               Calhoun  Chambers      Cherokee  ...    Park Platte   
2020-07-13      1.204511e-07  0.027778  8.549275e-08  ...     NaN    NaN   
2020-07-14      1.000001e-01  0.047222  4.274638e-08  ...     NaN    NaN   
2020-07-15      5.000003e-02  0.023611  2.137319e-08  ...     NaN    NaN   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2020-07-13          NaN      NaN        NaN   NaN   NaN   0.000411      NaN   
2020-07-14          NaN      NaN        NaN   NaN   NaN   0.025206      NaN   
2020-07-15          NaN      NaN        NaN   NaN   NaN   0.012603      NaN   

Province_State         
Admin2         Weston  
2020-07-13        NaN  
2020-07-14        NaN  
2020-07-15        NaN  

[3 rows x 3254 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200715,AK,1631,154462,NaN,32.0,NaN,NaN,NaN,0.0,...,156093,0,0,94f51a21cc1532dde7387df47013f567fb56d967,0,0,0,0,0,NaN
1,20200715,AL,59067,481982,NaN,1362.0,7291.0,NaN,941.0,NaN,...,541049,47,168,115a76b81b0c3c59d334b5de8180c2e10899dddc,0,0,0,0,0,NaN
2,20200715,AR,30297,365479,NaN,458.0,1948.0,NaN,NaN,94.0,...,395776,4,76,ce730ea246418cfcb8291076ec9ebfd46c643bfa,0,0,0,0,0,NaN
3,20200715,AS,0,816,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,4a0065a1c290d322d0dc631197c4e6cd8c714834,0,0,0,0,0,NaN
4,20200715,AZ,131354,604608,NaN,3493.0,6103.0,929.0,NaN,671.0,...,735962,97,161,cd92598667964b8af84aa921365c5bad41b04036,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200715,AK,1631.0,154462.0,NaN,32.0,NaN,NaN,NaN,0.0,...,156093,0,0,94f51a21cc1532dde7387df47013f567fb56d967,0,0,0,0,0,NaN
1,20200715,AL,59067.0,481982.0,NaN,1362.0,7291.0,NaN,941.0,NaN,...,541049,47,168,115a76b81b0c3c59d334b5de8180c2e10899dddc,0,0,0,0,0,NaN
2,20200715,AR,30297.0,365479.0,NaN,458.0,1948.0,NaN,NaN,94.0,...,395776,4,76,ce730ea246418cfcb8291076ec9ebfd46c643bfa,0,0,0,0,0,NaN
3,20200715,AS,0.0,816.0,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,4a0065a1c290d322d0dc631197c4e6cd8c714834,0,0,0,0,0,NaN
4,20200715,AZ,131354.0,604608.0,NaN,3493.0,6103.0,929.0,NaN,671.0,...,735962,97,161,cd92598667964b8af84aa921365c5bad41b04036,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-15,AK,1631,154462,NaN,32.0,NaN,NaN,NaN,0.0,NaN,...,156093,0,0,94f51a21cc1532dde7387df47013f567fb56d967,0,0,0,0,0,NaN
2020-07-15,AL,59067,481982,NaN,1362.0,7291.0,NaN,941.0,NaN,514.0,...,541049,47,168,115a76b81b0c3c59d334b5de8180c2e10899dddc,0,0,0,0,0,NaN
2020-07-15,AR,30297,365479,NaN,458.0,1948.0,NaN,NaN,94.0,279.0,...,395776,4,76,ce730ea246418cfcb8291076ec9ebfd46c643bfa,0,0,0,0,0,NaN
2020-07-15,AS,0,816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,4a0065a1c290d322d0dc631197c4e6cd8c714834,0,0,0,0,0,NaN
2020-07-15,AZ,131354,604608,NaN,3493.0,6103.0,929.0,NaN,671.0,NaN,...,735962,97,161,cd92598667964b8af84aa921365c5bad41b04036,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-07-15,AK,1631.0,154462.0,NaN,32.0,NaN,NaN,NaN,0.0,NaN,...,156093,0,0,94f51a21cc1532dde7387df47013f567fb56d967,0,0,0,0,0,NaN
2020-07-15,AL,59067.0,481982.0,NaN,1362.0,7291.0,NaN,941.0,NaN,514.0,...,541049,47,168,115a76b81b0c3c59d334b5de8180c2e10899dddc,0,0,0,0,0,NaN
2020-07-15,AR,30297.0,365479.0,NaN,458.0,1948.0,NaN,NaN,94.0,279.0,...,395776,4,76,ce730ea246418cfcb8291076ec9ebfd46c643bfa,0,0,0,0,0,NaN
2020-07-15,AS,0.0,816.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,816,0,0,4a0065a1c290d322d0dc631197c4e6cd8c714834,0,0,0,0,0,NaN
2020-07-15,AZ,131354.0,604608.0,NaN,3493.0,6103.0,929.0,NaN,671.0,NaN,...,735962,97,161,cd92598667964b8af84aa921365c5bad41b04036,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE